In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

In [2]:
import pandas as pd


train = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.data'))
test = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.test'))

In [3]:
ntrain = train.shape[0]
full = np.concatenate([train,test],axis=0)

c0 = full[:,0].astype(float).reshape([-1,1])
c1 = np.asarray(pd.get_dummies(full[:,1]))
c2 = full[:,2].astype(float).reshape([-1,1])
c3 = np.asarray(pd.get_dummies(full[:,3]))
c4 = full[:,4].astype(float).reshape([-1,1])
c5 = np.asarray(pd.get_dummies(full[:,5]))
c6 = np.asarray(pd.get_dummies(full[:,6]))
c7 = np.asarray(pd.get_dummies(full[:,7]))
c8 = np.asarray(pd.get_dummies(full[:,8]))
c9 = np.asarray(pd.get_dummies(full[:,9]))
c10 = full[:,10].astype(float).reshape([-1,1])
c11 = full[:,11].astype(float).reshape([-1,1])
c12 = full[:,12].astype(float).reshape([-1,1])
c13 = np.asarray(pd.get_dummies(full[:,13]))

label = np.zeros([full.shape[0],1])
label[full[:,14] == ' >50K',0] = 1
label[full[:,14] == ' >50K.',0] = 1

XFULL = np.concatenate([c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13],axis=1)
YFULL = label

xtrain = XFULL[0:ntrain,:]
ytrain = label[0:ntrain,0]

xtest = XFULL[ntrain:,:]
ytest = label[ntrain:,0]

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = wlgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['binary_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['binary']),
    'metric':hp.choice('metric',['binary_error']),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.5903657170639272, 'btype': 'In', 'extra_dims': 10, 'feature_fraction': 0.5987492171333759, 'lambda_l1': 0, 'lambda_l2': 1.3468198888337525e-05, 'learning_rate': 0.7150936118025001, 'metric': 'binary_error', 'min_data_in_leaf': 1.0, 'min_sum_hessian_in_leaf': 3.890409043805324e-05, 'num_leaves': 1.0, 'objective': 'binary'}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `fobj` in train.
Moving param `metric` to an feval.                     
[1]	valid_0's binary_error: 0.236241                   
[2]	valid_0's binary_error: 0.763759                   
[3]	valid_0's binary_error: 0.763759                   
[4]	valid_0's binary_error: 0.763759                   
[5]	valid_0's binary_error: 0.763759                   
[6]	valid_0's binary_error: 0.763759                   
[7]	valid_0's binary_error: 0.763759                   
[8]	valid_0's binary_error: 0.763759                   
[9]	valid_0's binary_error: 0.76

[17]	valid_0's binary_error: 0.186241                                             
[18]	valid_0's binary_error: 0.186241                                             
[19]	valid_0's binary_error: 0.186241                                             
[20]	valid_0's binary_error: 0.186241                                             
[21]	valid_0's binary_error: 0.186241                                             
[22]	valid_0's binary_error: 0.186241                                             
[23]	valid_0's binary_error: 0.186241                                             
[24]	valid_0's binary_error: 0.186241                                             
[25]	valid_0's binary_error: 0.186241                                             
[26]	valid_0's binary_error: 0.186241                                             
[27]	valid_0's binary_error: 0.186241                                             
[28]	valid_0's binary_error: 0.186241                                             
[29]

[7]	valid_0's binary_error: 0.133047                                             
[8]	valid_0's binary_error: 0.13231                                              
[9]	valid_0's binary_error: 0.131388                                             
[10]	valid_0's binary_error: 0.129484                                            
[11]	valid_0's binary_error: 0.129484                                            
[12]	valid_0's binary_error: 0.128686                                            
[13]	valid_0's binary_error: 0.12887                                             
[14]	valid_0's binary_error: 0.129423                                            
[15]	valid_0's binary_error: 0.128071                                            
[16]	valid_0's binary_error: 0.128194                                            
[17]	valid_0's binary_error: 0.128256                                            
[18]	valid_0's binary_error: 0.128071                                            
[19]	valid_0's b

Overwriting param `num_class`                                                     
Overwriting param `objective` while setting `fobj` in train.                      
Moving param `metric` to an feval.                                                
[1]	valid_0's binary_error: 0.142138                                              
[2]	valid_0's binary_error: 0.139128                                              
[3]	valid_0's binary_error: 0.137776                                              
[4]	valid_0's binary_error: 0.136732                                              
[5]	valid_0's binary_error: 0.136794                                              
[6]	valid_0's binary_error: 0.137469                                              
[7]	valid_0's binary_error: 0.136916                                              
[8]	valid_0's binary_error: 0.136425                                              
[9]	valid_0's binary_error: 0.135504                                              
[10]

[96]	valid_0's binary_error: 0.131818                                             
[97]	valid_0's binary_error: 0.131757                                             
[98]	valid_0's binary_error: 0.131757                                             
[99]	valid_0's binary_error: 0.131941                                             
[100]	valid_0's binary_error: 0.13188                                             
{'bagging_fraction': 0.5557258221784855, 'btype': 'I', 'extra_dims': 6, 'feature_fraction': 0.504444201991018, 'lambda_l1': 0, 'lambda_l2': 7.0530243726541055, 'learning_rate': 0.04538293643973183, 'metric': 'binary_error', 'min_data_in_leaf': 159.0, 'min_sum_hessian_in_leaf': 1.8300576604866534e-07, 'num_leaves': 52.0, 'objective': 'binary'}
Overwriting param `num_class`                                                     
Overwriting param `objective` while setting `fobj` in train.                      
Moving param `metric` to an feval.                                        

...

[5]	valid_0's binary_error: 0.135811                                               
[6]	valid_0's binary_error: 0.135197                                               
[7]	valid_0's binary_error: 0.133047                                               
[8]	valid_0's binary_error: 0.132064                                               
[9]	valid_0's binary_error: 0.130405                                               
[10]	valid_0's binary_error: 0.130098                                              
[11]	valid_0's binary_error: 0.13016                                               
[12]	valid_0's binary_error: 0.129607                                              
[13]	valid_0's binary_error: 0.128563                                              
[14]	valid_0's binary_error: 0.12801                                               
[15]	valid_0's binary_error: 0.127334                                              
[16]	valid_0's binary_error: 0.127518                                       

Overwriting param `num_class`                                                      
Overwriting param `objective` while setting `fobj` in train.                       
Moving param `metric` to an feval.                                                 
[1]	valid_0's binary_error: 0.148342                                               
[2]	valid_0's binary_error: 0.148219                                               
[3]	valid_0's binary_error: 0.146867                                               
[4]	valid_0's binary_error: 0.144287                                               
[5]	valid_0's binary_error: 0.143059                                               
[6]	valid_0's binary_error: 0.143305                                               
[7]	valid_0's binary_error: 0.141708                                               
[8]	valid_0's binary_error: 0.141769                                               
[9]	valid_0's binary_error: 0.140786                                        

[95]	valid_0's binary_error: 0.125184                                              
[96]	valid_0's binary_error: 0.125                                                 
[97]	valid_0's binary_error: 0.125246                                              
[98]	valid_0's binary_error: 0.125061                                              
[99]	valid_0's binary_error: 0.125246                                              
[100]	valid_0's binary_error: 0.125369                                             
{'bagging_fraction': 0.5987397559452049, 'btype': 'Rn', 'extra_dims': 14, 'feature_fraction': 0.5914244228699912, 'lambda_l1': 0, 'lambda_l2': 1.0879344631190209e-05, 'learning_rate': 0.006595655069834732, 'metric': 'binary_error', 'min_data_in_leaf': 6.0, 'min_sum_hessian_in_leaf': 6.893485096285418e-07, 'num_leaves': 37.0, 'objective': 'binary'}
Overwriting param `num_class`                                                      
Overwriting param `objective` while setting `fobj` in train.  

[85]	valid_0's binary_error: 0.124631                                              
[86]	valid_0's binary_error: 0.124754                                              
[87]	valid_0's binary_error: 0.12457                                               
[88]	valid_0's binary_error: 0.124693                                              
[89]	valid_0's binary_error: 0.124631                                              
[90]	valid_0's binary_error: 0.124816                                              
[91]	valid_0's binary_error: 0.124631                                              
[92]	valid_0's binary_error: 0.124631                                              
[93]	valid_0's binary_error: 0.124509                                              
[94]	valid_0's binary_error: 0.124386                                              
[95]	valid_0's binary_error: 0.124386                                              
[96]	valid_0's binary_error: 0.124509                                       

[75]	valid_0's binary_error: 0.15043                                               
[76]	valid_0's binary_error: 0.150369                                              
[77]	valid_0's binary_error: 0.150491                                              
[78]	valid_0's binary_error: 0.150123                                              
[79]	valid_0's binary_error: 0.149939                                              
[80]	valid_0's binary_error: 0.15                                                  
[81]	valid_0's binary_error: 0.149754                                              
[82]	valid_0's binary_error: 0.149877                                              
[83]	valid_0's binary_error: 0.150184                                              
[84]	valid_0's binary_error: 0.14957                                               
[85]	valid_0's binary_error: 0.149631                                              
[86]	valid_0's binary_error: 0.149693                                       

In [5]:
print(best_val)
print(space_eval(spc, best))

0.12285012285012285
{'bagging_fraction': 0.6581278555915002, 'btype': 'Rn', 'extra_dims': 10, 'feature_fraction': 0.5777448269808154, 'lambda_l1': 0.0003958936252155316, 'lambda_l2': 0.0035424985582939088, 'learning_rate': 0.0267068555896055, 'metric': 'binary_error', 'min_data_in_leaf': 6.0, 'min_sum_hessian_in_leaf': 0.0011268057014953772, 'num_leaves': 12.0, 'objective': 'binary'}


In [6]:
(0.12383292383292384 - 0.12285012285012285)/0.12383292383292384

0.007936507936507957